# Game 3 - Hospital Resources

Skillfully allocating medical resources during a pandemic can save lives. Unexpected and extreme medical events not only disrupt hospitals' normal routines and protocol, but strain resources such as trained staff, space and pharmaceuticals, in addition to PPE and life-saving medical equipment. Determining how to most effectively allocate these resources is paramount to assuring the wellbeing of Sailors far from port.

In this scenario, you will oversee the medical bay of the USS GHOST where COVID-19 has taken hold of the crew. Your task is to create an algorithm to deploy the ship's available resources in a manner that will ensure as many Sailors as possible recover from the virus.

## brainstorming:
- reinforcement learning?

In [419]:
import pandas as pd
import numpy as np
from itertools import combinations

* +30 10 ventilators (reusable)
* +20 10 oxygen masks (reusable)
    * cannot be combined with ventilator
* +15 10 plasma
* +30 7 remdesivir
* +25 20 Dexamethasone
* +15 10 casirivimab
* +10 17 chloroquine
* 30 beds
    * bed boosts health 5 points
    * bed allocation required for other treatments
* final health score = final health + bed bonus + rate of decline + treatment bonuses
* penalty of -100 if any sailor final health is 0


In [424]:
sailors_df = pd.read_csv("day2data.csv", header=1, index_col=0)
# sailors_df.drop("Unnamed: 3", axis=1, inplace=True)
resource_list = ["Bed", "Ventilator", "Remdesivir", "Dexamethasone", "Plasma", "Casirivimab", "Supplemental oxygen", "Chloroquine"]
for resource in resource_list:
    sailors_df[resource] = 0
columns = ["resource", "count", "amt_used", "reusable", "health_bonus"]
data = np.array([["Bed", 30, 0, True, 5],
          ["Ventilator", 10, 0, True, 30],
          ["Supplemental oxygen", 10, 0, True, 20],
          ["Plasma", 10, 0, False, 15],
          ["Remdesivir", 7, 0, False, 30],
          ["Dexamethasone", 20, 0, False, 25],
          ["Casirivimab", 10, 0, False, 15],
          ["Chloroquine", 17, 0, False, 10]])
resources_df = pd.DataFrame(data=data, columns=columns)
for col in ["count", "amt_used", "health_bonus"]:
    resources_df[col] = pd.to_numeric(resources_df[col])
print(resources_df,"\n")
sailors_df

              resource  count  amt_used reusable  health_bonus
0                  Bed     30         0     True             5
1           Ventilator     10         0     True            30
2  Supplemental oxygen     10         0     True            20
3               Plasma     10         0    False            15
4           Remdesivir      7         0    False            30
5        Dexamethasone     20         0    False            25
6          Casirivimab     10         0    False            15
7          Chloroquine     17         0    False            10 



,Health,Rate of health decline,Unnamed: 3,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine
ID,,,,,,,,,,,
21,54,-1,NaN,0,0,0,0,0,0,0,0
22,23,-20,NaN,0,0,0,0,0,0,0,0
23,8,-40,NaN,0,0,0,0,0,0,0,0
24,45,-41,NaN,0,0,0,0,0,0,0,0
25,100,-44,NaN,0,0,0,0,0,0,0,0
26,87,-45,NaN,0,0,0,0,0,0,0,0
27,65,-50,NaN,0,0,0,0,0,0,0,0
28,79,-60,NaN,0,0,0,0,0,0,0,0
29,56,-65,NaN,0,0,0,0,0,0,0,0


In [425]:
def use_resource(sailors_df, resources_df, resource, indices):
    """
    resource = resource name as string
    indices = Pandas index list
    Ex. for single indices do [df.index[position]]
    Ex. for range of values do df.index[start:end]
    """
    bonus_dict = {}
    for r, bonus in zip(resources_df["resource"], resources_df["health_bonus"]):
        bonus_dict[r] = int(bonus)

    sailors_df.loc[indices, resource] = bonus_dict[resource]
    sailors_df.loc[indices, "Total"] += bonus_dict[resource]
    resources_df.loc[resources_df["resource"] == resource, "amt_used"] = len(indices)
    resources_df.loc[resources_df["resource"] == resource, "count"] -= len(indices)

def allocate_reusable(sailors_df, resources_df):    
    add_beds = sailors_df.index[:30]
    use_resource(sailors_df, resources_df, "Bed", add_beds)
    
    add_ventilators = sailors_df.index[:10]
    use_resource(sailors_df, resources_df, "Ventilator", add_ventilators)
    
    oxymask_candidates = sailors_df[(sailors_df["Ventilator"]==0) & (sailors_df["Bed"]>0)]
    add_oxymask = oxymask_candidates.index[:10]
    use_resource(sailors_df, resources_df, "Supplemental oxygen", add_oxymask)

def ded(sailors_df, resources_df):
    def get_bonus(tups):
        total = 0
        for name, bonus in tups:
            total += bonus
        return total
    def get_names(tups):
        names = []
        for name, bonus in tups:
            names.append(name)
        return names

    mindict = {}
    minlist = []
    dying = sailors_df.loc[sailors_df["Total"] < 0]
    nonreusable = resources_df[(resources_df["reusable"]=="False") & (resources_df["count"] > 0)]
    tups = list(zip(nonreusable["resource"], nonreusable["health_bonus"]))
    for i in range(len(dying)):
        y = dying.iloc[i]["Total"]
        comb_length = 1
        while len(minlist) == 0 and comb_length < len(tups):
            for comb in combinations(tups, comb_length):
                x = y + get_bonus(comb)
                mindict[x] = get_names(comb)
            minlist = [k for k in mindict.keys() if k > 0]
            comb_length += 1
        m = min(minlist)
        for resource in mindict[m]:
            use_resource(sailors_df, resources_df, resource, [sailors_df.index[i]])
        mindict = {}
        minlist = []

def allocation(sailors_df, resources_df):
    resources_df = resources_df.copy()
    sailors_df = sailors_df.copy()
    sailors_df["Total"] = sailors_df["Health"]+sailors_df["Rate of health decline"]
    sailors_df.sort_values(by=["Total"], inplace=True)
    
    allocate_reusable(sailors_df, resources_df)
    ded(sailors_df, resources_df)
    
    return sailors_df, resources_df

def calc_final_score(allocation_df):
    final_health = allocation_df["Total"].apply(lambda x: 100 if x > 100 else x)
    final_health = final_health.apply(lambda x: -100 if x < 1 else x)
    return final_health.sum()


In [429]:
a_df, r_df = allocation(sailors_df, resources_df)
a_df

,Health,Rate of health decline,Unnamed: 3,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Total
ID,,,,,,,,,,,,
39,9,-92,NaN,5,30,30,25,0,0,0,0,7
47,34,-99,NaN,5,30,0,25,0,0,0,10,5
35,15,-80,NaN,5,30,0,25,0,0,0,10,5
45,45,-99,NaN,5,30,0,25,0,0,0,0,6
36,34,-82,NaN,5,30,0,0,0,15,0,0,2
44,59,-99,NaN,5,30,0,0,0,0,0,10,5
23,8,-40,NaN,5,30,0,0,0,0,0,0,3
37,60,-85,NaN,5,30,0,0,0,0,0,0,10
33,56,-75,NaN,5,30,0,0,0,0,0,0,16


(('Plasma', 15),)
(('Remdesivir', 30),)
(('Dexamethasone', 25),)
(('Casirivimab', 15),)
(('Chloroquine', 10),)


0